## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/validation_data/mouse_pair4.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# segment paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/validation_data/mouse_pair4_segmentation.h5'
hdf5_file_seg = h5py.File(data_path, 'r')
fix_seg = hdf5_file_seg['/fixed'][...]
mov_seg = hdf5_file_seg['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([0.4, 0.1625, 0.1625])
mov_spacing = np.array([0.4, 0.1625, 0.1625])

# # # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)
# nrrd.write('./fix_seg.nrrd', fix_seg.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov_seg.nrrd', mov_seg.transpose(2,1,0), compression_level=2)

## Masking
---

### fixed

In [2]:
# tools for coarse whole brain segmentation
from bigstream import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# fix_skip = fix[::1, ::4, ::4]
# skip_spacing = fix_spacing * [1, 4, 4]
# fix_mask = level_set.foreground_segmentation(
#     fix_skip, skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,2],
#     lambda2=37.0,
# )

# # smooth then enlarge mask
# fix_mask = binary_closing(fix_mask, np.ones((5,5,5))).astype(np.uint8)
# fix_mask = binary_dilation(fix_mask, np.ones((5,5,5))).astype(np.uint8)
# fix_mask = zoom(fix_mask, np.array(fix.shape) / fix_skip.shape, order=0)

# # write result
# nrrd.write('./fix_mask.nrrd', fix_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
fix_mask, _ = nrrd.read('./fix_mask.nrrd')
fix_mask = fix_mask.transpose(2,1,0)

### moving

In [3]:
# tools for coarse whole brain segmentation
from bigstream import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# mov_skip = mov[::1, ::4, ::4]
# skip_spacing = mov_spacing * [1, 4, 4]
# mov_mask = level_set.foreground_segmentation(
#     mov_skip, skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,2],
#     lambda2=37.0,
# )

# # enlarge and smooth mask
# mov_mask = binary_closing(mov_mask, np.ones((5,5,5))).astype(np.uint8)
# mov_mask = binary_dilation(mov_mask, np.ones((5,5,5))).astype(np.uint8)
# mov_mask = zoom(mov_mask, np.array(mov.shape) / mov_skip.shape, order=0)

# # write result
# nrrd.write('./mov_mask.nrrd', mov_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
mov_mask, _ = nrrd.read('./mov_mask.nrrd')
mov_mask = mov_mask.transpose(2,1,0)

## Alignment
---

### affine

In [2]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':0.5,
    'shrink_factors':(8,4,2,1),
    'smooth_sigmas':(4.,4.,2.,1.),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
}

steps = [('affine', common_kwargs),]

In [5]:
%%time

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

LEVEL:  0  ITERATION:  0  METRIC:  -0.28809333762395906
LEVEL:  0  ITERATION:  1  METRIC:  -0.2926085781927745
LEVEL:  0  ITERATION:  2  METRIC:  -0.2961261475958429
LEVEL:  0  ITERATION:  3  METRIC:  -0.2990257909444468
LEVEL:  0  ITERATION:  4  METRIC:  -0.30159242183374463
LEVEL:  0  ITERATION:  5  METRIC:  -0.3039608353006222
LEVEL:  0  ITERATION:  6  METRIC:  -0.3065526008948673
LEVEL:  0  ITERATION:  7  METRIC:  -0.30618933868551995
LEVEL:  0  ITERATION:  8  METRIC:  -0.31105132401554236
LEVEL:  0  ITERATION:  9  METRIC:  -0.31356995017926526
LEVEL:  0  ITERATION:  10  METRIC:  -0.3139148671648455
LEVEL:  0  ITERATION:  11  METRIC:  -0.3142351975879116
LEVEL:  0  ITERATION:  12  METRIC:  -0.3144409564302674
LEVEL:  0  ITERATION:  13  METRIC:  -0.3146403639323715
LEVEL:  0  ITERATION:  14  METRIC:  -0.3147892418157424
LEVEL:  0  ITERATION:  15  METRIC:  -0.3148864434555893
LEVEL:  0  ITERATION:  16  METRIC:  -0.314931566639232
LEVEL:  0  ITERATION:  17  METRIC:  -0.314933558014978

In [6]:
# apply transform and save results
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# apply transform and save results
affine_aligned_seg = apply_transform(
    fix_seg, mov_seg,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
    interpolate_with_nn=True,
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)
nrrd.write('./affine_seg.nrrd', affine_aligned_seg.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

In [2]:
from bigstream.utility import matrix_to_displacement_field

# # load affine, convert to displacement, convert to voxel units
# affine = np.loadtxt('./affine.mat')
# deform = matrix_to_displacement_field(affine, fix.shape, spacing=fix_spacing)
# deform = deform / fix_spacing

deform = np.zeros(fix.shape + (3,), dtype=np.float32)

# save as h5 file
with h5py.File('./mouse_val.h5', 'w') as f:
    dset = f.create_dataset('pair4', deform.shape, deform.dtype)
    dset[...] = deform